<a href="https://colab.research.google.com/github/narendra974/insidedeeplearning/blob/main/PyTorchBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This NoteBook illustrates the examples given in chapter 1 of Inside Deep Learning Book.